## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import numpy as np
import re

## Data

In [2]:
session_cookie = quote("UyfbwAuNgiGBW0Kls24zFdDxjkJhCZoFxw5Vy76OKto/YwnvlKRF1ZaqmqE9foWEEx1SamHVsDzGAc0aGM6uGhmdRyny6esb962ttVGDF89T1OsQwHq4NVwuasvgg70NowAMhqKnvek86bhM4J5o/Oe7xXR/VHKn1rIs62jBCmS/0YZwVZWyoBMrDYbw/IP6lk6Y+hU8Fx6qAk5Mx+9p0D4AdIPhypmERFU56ii6DZoUcL9ixUfbo8gNpbDF+JU0n/3o/46YfTDlPJDVy3HkuzA/Q9GwzkiRYvy0a2LMlP7buFEdIwJcom+S38CH3axZUJv0pMH4DyLVTLv5Y9idMcBJH1gq69xpIN+sqQ==--Pa3YFPf36fSY1YAh--tIcJOkufJL4qrIwDtEiVeQ==", safe = "")
#session_cookie = quote(input("Copy LW x AOC session cookie here: "))
jinglebet_url = "https://www.41051.com/xmaslyrics/jingle.html"
rudolphabet_url = "https://www.41051.com/xmaslyrics/rudolph.html"
alphabet = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzÇàéè0123456789 .!?,;:'-#/\()")

## Temp

In [3]:
lines = [line.strip("\n") for line in open("output.txt").readlines()]

## Functions

### AOC Interface

In [4]:
def get_user_rankings(user_id):
    '''
    Get daily rankings from day 1 through 25 for user_id.
    Returns two separate lists for parts 1 and 2.
    '''
    # Settings
    cookies = {
        "_lewagon_aoc_session": session_cookie,
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Accept": "text/html, application/xhtml+xml",
        "Accept-Language": "de,en-US;q=0.7,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://aoc.lewagon.community/",
        "Turbolinks-Referrer": "https://aoc.lewagon.community/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
    }
    url = "https://aoc.lewagon.community/stats/users/"
    # Initiate lists
    part_one = []
    part_two = []
    # Get data from server
    response = requests.get(f"{url}{user_id}", headers=headers, cookies=cookies)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.findAll("table")[0]
    # Iterate over rows in table
    for idx, row in enumerate(table.findAll("tr")):
        if idx > 1:
            cells = row.findAll("td")
            part_one.append(cells[1].get_text())
            part_two.append(cells[4].get_text())
    # Reverse order and turn strings into numbers
    part_one = [int(c) if c.isnumeric() else -1 for c in part_one[::-1]]
    part_two = [int(c) if c.isnumeric() else -1 for c in part_two[::-1]]
    return part_one, part_two
print(get_user_rankings(59))

([55, 235, 76, 86, 61, 77, 49, 12, 86, 56, 45, 38, 68, 57, 29, 33, 7, 17, 7, 23, 25, 21, 15, 5, 14], [53, 228, 75, 70, 49, 118, 44, 16, 72, 54, 44, 32, 61, 43, 31, 26, 7, 13, 4, 19, 16, 4, 6, 3, 13])


### Data Import

In [5]:
def extract_inputs(line):
    '''
    Splits each line string into user ID (int) and message (str)
    '''
    user_id, message = line.split(": ", 1)
    return int(user_id), message

### Product

In [6]:
def prod(key):
    '''
    Returns product of key elements
    '''
    a = 1
    for i in key:
        a *= i
    return a

### Ternary Conversion

In [7]:
def get_ternary(cipher):
    '''
    Returns ternary representation of number
    '''
    if cipher == 0:
        return "0"
    nums = []
    while cipher:
        cipher, r = divmod(cipher, 3)
        nums.append(str(r))
    return "".join(reversed(nums))

### String Shuffle

In [8]:
def shuffle_string(message, ter_cipher):
    '''
    Shuffles string according to rules
    '''
    for idx, val in enumerate(ter_cipher[::-1]):
        if val == "0":
            message = message[::-1]
        elif val == "1":
            message = message[1:] + message[0]
        else:
            shift = (3**idx) % len(message)
            message = message[-shift:] + message[:-shift]
    return message

### Prime Factors

In [9]:
def get_prime_factor_sum(cipher):
    '''
    Break number into prime factors and return sum, or sum + 5 if only two factors
    '''
    factors = []
    f = 2
    while f * f <= cipher:
        if cipher % f:
            f += 1
        else:
            cipher //= f
            factors.append(f)
    if cipher > 1:
        factors.append(cipher)
    return np.sum(factors) + (5 if len(factors) == 1 else 0)

### Jinglebet/Rudolphabet Shifting

In [10]:
def get_alphabet(url):
    '''
    Get re-sorted alphabet
    '''
    # Get lyrics from web
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    lyrics = re.sub("[^a-zA-Z]+", "", "".join(soup.find("span", {"class": "bodytext"}).get_text().split()))
    # Add relevant letters in order they appear
    new_alphabet = ""
    for char in lyrics:
        if char not in new_alphabet:
            new_alphabet += char
            alphabet.pop(alphabet.index(char))
    # Add remaining letters that did not feature in the lyrics
    new_alphabet += "".join(alphabet)
    return new_alphabet

def shift_alphabet(shift):
    '''
    Shift alphabet required number of characters
    '''
    shift = shift % len(new_alphabet)
    shifted_alphabet = new_alphabet[-shift:] + new_alphabet[:-shift]
    return shifted_alphabet

def shift_message(message, shifted_alphabet):
    return "".join([new_alphabet[shifted_alphabet.index(char)] for char in message])

## Part 1

In [11]:
print(prod(get_user_rankings(1)[1]))

9913814436643373168640000000


In [ ]:
9913814436643373168640000000
9913814436643373168640000000

## Part 2

In [12]:
# Obtain new alphabet
new_alphabet = get_alphabet(rudolphabet_url)

# Iterate over messages
for line in lines:
    # Extract data
    user_id, message = extract_inputs(line)
    # Find daily rankings
    rankings = get_user_rankings(user_id)
    # Get cipher
    cipher = int(str(abs(prod(rankings[1]) + user_id * (1 if user_id % 2 == 0 else -1)))[::3])
    # Convert into ternary
    ter_cipher = get_ternary(cipher)
    # Shuffle string
    message = shuffle_string(message, ter_cipher)
    # Obtain shifted alphabet
    shifted_alphabet = shift_alphabet(get_prime_factor_sum(cipher))
    # Decode message
    message = shift_message(message, shifted_alphabet)
    print(f"{user_id}: {message}")

391: Dear Pilou, we can't wait to host you in BXL, town of your batch - the best - #1970. Love!
312: Thank you Pilou for giving me a reason for getting back in touch with my batchmates... and for making me face the fact that I'm still a super mediocre programmer
549: Hi Pilou, thank you for taking the time to build this platform for AOC '21. It's been really good fun and a great learning opportunity as well. Thanks
710: Vive le Pilou, à bas le poisson!
507: Thank you Pilou, for all of your work to make Le Wagon x AoC such a great event!!! Looking forward to next year!
17: Many thanks for introducing us to AoC (I had no idea it was a thing before). I learned so much, mainly about recursion, complexity and concepts that come up a lot with it (BFS, DFS, Dijkstra etc). Wishing you and yours eternal joy and happiness and I look forward to next year's AoC
305: thanks pil0u for this event, it was awesome to meet the growing community, I can't wait for the next edition !
660: Merci Pilou pour 